In [1]:
from safetensors import safe_open
from transformers.trainer_utils import get_last_checkpoint
from glob import glob
from transformers import GptOssForCausalLM, AutoModelForCausalLM, AutoTokenizer, Mxfp4Config
from peft import PeftModel
import torch
from multiprocess import Pool
import itertools
import os

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))
        
torch.set_grad_enabled(False)

/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [2]:
def loop(folders):
    folders, index = folders
    os.environ['CUDA_VISIBLE_DEVICES'] = str(index)
    tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
    for folder in folders:
        print(folder)
        total_rank = int(folder.split('-r')[-1].split('-')[0])
        tensors = {}
        f = os.path.join(get_last_checkpoint(folder), 'weight.pt')
        with safe_open(f, framework="pt", device='cpu') as f:
            for k in f.keys():
                tensors[k] = f.get_tensor(k)
    
        model_kwargs = dict(
            attn_implementation="kernels-community/vllm-flash-attn3",
            torch_dtype=torch.bfloat16, 
            use_cache=True, 
        )
        model = AutoModelForCausalLM.from_pretrained("unsloth/gpt-oss-20b-BF16", **model_kwargs).cuda()
    
        state_dict = model.state_dict()
    
        top_k = model.config.num_experts_per_tok
        r = total_rank // top_k
        alpha = (total_rank * 2) // top_k
        merge_scale = alpha / r
        
        for i in range(model.config.num_hidden_layers):
            if f'model.layers.{i}.mlp.experts.lora_gate_up_A.e.weight' in tensors:
                W = state_dict[f'model.layers.{i}.mlp.experts.gate_up_proj']
                A = tensors[f'model.layers.{i}.mlp.experts.lora_gate_up_A.e.weight'].to(W.device)
                B = tensors[f'model.layers.{i}.mlp.experts.lora_gate_up_B.e.weight'].to(W.device)
                for k in range(model.config.num_local_experts):
                    a = A[k].reshape(-1, r)
                    b = B[k].reshape(r, -1)
            
                    m = torch.matmul(a, b) * merge_scale
                    W[k] += m.to(W.dtype)
        
            if f'model.layers.{i}.mlp.experts.lora_down_B.e.weight' in tensors:
                W = state_dict[f'model.layers.{i}.mlp.experts.down_proj']
                A = tensors[f'model.layers.{i}.mlp.experts.lora_down_A.e.weight'].to(W.device)
                B = tensors[f'model.layers.{i}.mlp.experts.lora_down_B.e.weight'].to(W.device)
                for k in range(model.config.num_local_experts):
                    a = A[k].reshape(-1, r)
                    b = B[k].reshape(r, -1)
            
                    m = torch.matmul(a, b) * merge_scale
                    W[k] += m.to(W.dtype)
    
        keys = tensors.keys()
        keys_lora = sorted(list(set([k.split('.lora')[0] for k in keys if '.self_attn.' in k])))
        for k in keys_lora:
            k_ori = k + '.weight'
            post_A = '.lora_A.e.weight'
            post_B = '.lora_B.e.weight'
            A = k + post_A
            B = k + post_B
            W = state_dict[k_ori]
            A = tensors[A].type(W.dtype).to(W.device)
            B = tensors[B].type(W.dtype).to(W.device)
            m = torch.matmul(A.t(), B.t()) * 2.0
            W += m.T.to(W.dtype)
    
        model.save_pretrained(f'{os.path.split(folder)[1]}-merged')
        tokenizer.save_pretrained(f'{os.path.split(folder)[1]}-merged')
    
        del model

In [3]:
folders = glob('/root/malaysian-reasoning-20b-lora-r*experts')

In [4]:
multiprocessing(folders, loop, cores=4, returned=False)

/root/malaysian-reasoning-20b-lora-r128-experts
/root/malaysian-reasoning-20b-lora-r128-selected-experts/root/malaysian-reasoning-20b-lora-r256-experts

/root/malaysian-reasoning-20b-lora-r256-selected-experts


`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 447.91it/s]
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or dire

/root/malaysian-reasoning-20b-lora-r32-selected-experts


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 455.82it/s]


/root/malaysian-reasoning-20b-lora-r64-selected-experts


...

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 455.36it/s]


/root/malaysian-reasoning-20b-lora-r32-experts


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 438.42it/s]


/root/malaysian-reasoning-20b-lora-r512-experts


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 465.56it/s]


In [5]:
print('done')

done
